$$
    \require{cancel}
    \newcommand{\genericdel}[3]{%
      \left#1#3\right#2
    }
    \newcommand{\del}[1]{\genericdel(){#1}}
    \newcommand{\sbr}[1]{\genericdel[]{#1}}
    \newcommand{\cbr}[1]{\genericdel\{\}{#1}}
    \newcommand{\abs}[1]{\genericdel||{#1}}
    \DeclareMathOperator*{\argmin}{arg\,min}
    \DeclareMathOperator*{\argmax}{arg\,max}
    \DeclareMathOperator{\Pr}{\mathbb{p}}
    \DeclareMathOperator{\E}{\mathbb{E}}
    \DeclareMathOperator{\Ind}{\mathbb{I}}
    \DeclareMathOperator{\var}{\mathbb{V}}
    \DeclareMathOperator{\mse}{MSE}
    \DeclareMathOperator{\cov}{Cov}
    \DeclareMathOperator{\corr}{Corr}
    \DeclareMathOperator{\invchi}{\mathrm{Inv-\chi}^2}
    \newcommand{\effect}{\mathrm{eff}}
    \newcommand{\xtilde}{\widetilde{X}}
    \DeclareMathOperator{\normal}{\mathcal{N}}
    \DeclareMathOperator{\unif}{Uniform}
    \DeclareMathOperator{\GP}{\mathcal{GP}}
    \newcommand{\T}{\mathrm{T}}
    \newcommand{\Tn}{\T_{n}}
    \newcommand{\Tx}{\T_{x}}
    \newcommand{\station}[1]{\mathrm{station}\sbr{#1}}
    \newcommand{\xvec}{\mathbf{x}}
    \newcommand{\hvec}{\mathbf{h}}
    \newcommand{\indep}{\perp}
    \newcommand{\iid}{iid}
    \newcommand{\trans}{^{\intercal}}
    \newcommand{\sigmaf}{\sigma_{\mathrm{GP}}}
    \newcommand{\sigman}{\sigma_{\epsilon}}
    \newcommand{\degreeC}{^\circ \mathrm{C}}
    \newcommand{\miss}{\mathrm{miss}}
    \newcommand{\obs}{\mathrm{nearby}}
    \DeclareMathOperator*{\softmax}{smoothmax}
    \DeclareMathOperator*{\softmin}{smoothmin}
    \newcommand{\error}{\mathrm{err}}
    \newcommand{\hour}{ \mathtt{hr} }
    \DeclareMathOperator{\kSESE}{k_{\mathtt{SExSE}}}
    \DeclareMathOperator{\kdiurn}{k_{\mathtt{SESE\_24}}}
    \DeclareMathOperator{\ksumprod}{k_{\mathtt{sumprod}}}
    \newcommand{\iday}{\mathtt{day}}
    \newcommand{\dayset}[1]{\cbr{i}_{#1}}
    \newcommand{\discrepancy}{\delta}
    \newcommand{\Xmax}{X_{\max}}
    \newcommand{\Xmin}{X_{\min}}
    \newcommand{\Fcond}{F_{X \mid \Xmax,\Xmin}}
    \newcommand{\pxx}[2]{\Pr{}_{#1#2}}
    \newcommand{\pij}{\pxx{i}{j}}
    \newcommand{\pisum}{\pxx{i}{\bullet}}
    \newcommand{\psumj}{\pxx{\bullet}{j}}
    \newcommand{\eqlabel}[1]{\label{#1}}
    \newcommand{\thetagp}{\theta_{\GP}}
    \newcommand{\mean}{m}
    \DeclareMathOperator{\midpoint}{middle}
    \newcommand{\window}{\mathtt{win}}
    \DeclareMathOperator{\buffer}{\mathtt{buff}}
    \newcommand{\best}{\mathtt{best}}
    \DeclareMathOperator{\timespan}{\Delta}
$$

**Executive summary**

* In 2015, the average temperature at Waterloo Municipal Airport was $9.497\,\degreeC$.
* Extracting daily $\Tn/\Tx$ at 17:00 UTC each day and taking their mean yields an estimate of the average temperature of $9.716\,\degreeC$.
* Taking the mean of the imputed temperatures yields a posterior distribution of the average temperature of $9.474\,\degreeC \pm 0.020$.
* That's pretty good!

**Background**

In the paper presenting the temperature imputations approach to the hour of measurement bias, we demonstrated our ability to “recover” summary statistics such as the average maximum temperature.
We focused on statistics such as the average daily maximum temperature, that are deterministic functions of the daily extrema, which are in turn functions of the true temperatures and hour of measurement.
To formalize this a little bit, let $\T_\miss$ be the time series of temperatures at the location of interest, and $\hour$ be the hour of measurement.
Then we can write the daily temperature extrema as functions $\Tn\del{\T_\miss, \hour}$ and $\Tx\del{\T_\miss, \hour}$.
We additionally used information from temperature time series at nearby airport, which we denote $\T_\obs$.
The paper shows how to draw imputations of $\T_\miss$ from its posterior distribution $\T_\miss \mid \Tn, \Tx, \T_\obs, \hour$.
The summary statistics of interest are of the form $h\del{\Tn, \Tx}$, which we can expand to $h\del{\Tn\del{\T_\miss, \hour}, \Tx\del{\T_\miss, \hour}}$.
Therefore the value of the summary statistic indirectly depends on the hour of measurement.
In the paper, we show we can infer reasonably well what the statistic *would have been*, had we made the measurements at a different time $\hour'$.
That is, we successfully obtain samples from the posterior distribution
\begin{equation}
    h\del{\Tn\del{\T_\miss, \hour'}, \Tx\del{\T_\miss, \hour'}}
    \mid
    \Tn\del{\T_\miss, \hour}, \Tx\del{\T_\miss, \hour}, \T_\obs, \hour
\end{equation}
simply by applying the function $h\del{\Tn\del{\T_\miss, \hour'}, \Tx\del{\T_\miss, \hour'}}$
to the imputations of $\T_\miss$ we had previously obtained.

**New Problem**

In *this* document, we are interested in the year's average temperature, which is a slightly different kind of summary statistic.
Instead of being of function of $\Tn$ and $\Tx$, it is a function directly of $\T_\miss$, and does not depend on $\hour$, so we can write it simply as $\mean\del{\T_\miss}$.
However, since $\T_\miss$ is not available, the average temperature is commonly *estimated approximately* by a statistic of the first form, the mean of the minima and maxima:
\begin{equation}
    \mean\del{\T_\miss} \approx h\del{\Tn, \Tx} = \del{\overline{\Tn} + \overline{\Tx}} \big/ 2\,.
\end{equation}
The estimate depends on $\hour$ but the estimand does not.
What we propose is simply to once again obtain the posterior distribution of $\mean\del{\T_\miss}$ by applying the $\mean$ function to the imputations, which gives us samples from
\begin{equation}
    \mean\del{\T_\miss}
    \mid
    \Tn\del{\T_\miss, \hour}, \Tx\del{\T_\miss, \hour}, \T_\obs, \hour
\end{equation}

**True mean temperature at Waterloo Airport**

![\label{fig:histogram_time_diff}Histogram of time differences](../figures/histogram_time_diff.png)

Because I already have imputations, it was easiest to apply this idea to the Waterloo Municipal Airport.
In that case I actually have access to hourly measurements, so I can compute the true $\mean\del{\T_\miss}$.
The time series is provided as $N=12,695$ measurements $\T_{\miss,i}$, $i=1,\dotsc,N$ made at times $t_i$.
I refer to these as hourly measurements, but in reality the time series is irregular, with median difference of 54 minutes, mean difference 41.4 minutes, minimum difference 1 minute, and maximum difference 120 minutes (histogram in Figure \ref{fig:histogram_time_diff}).
For this reason, we can't simply average the temperature measurements, we should weight each observation by the amount of time it occupies.
This gives weight
\begin{equation}
w_i = \begin{cases}
    \frac{1}{2}\del{t_{i+1}-t_i} & ,i=1 \\
    \frac{1}{2}\del{t_{i+1}-t_{i-1}} & ,i=2,\dotsc,N-1 \\
    \frac{1}{2}\del{t_{i}-t_{i-1}} & ,i = N
    \end{cases}
\end{equation}
to the $i^\text{th}$ observation $\T_{\miss,i}$.
Another way to write this, which turns out to be notationally more convenient, is to weight the temperature midpoints
\begin{equation}
    \midpoint\del{\T_{\miss}, i} \equiv \del{ \T_{\miss,i+1} + \T_{\miss,i} } \big/ 2 \,,i=1,\dotsc,N-1
\end{equation}
by the time differences
\begin{equation}
    \Delta_i = t_{i+1}-t_i\,.
\end{equation}
The weighted mean can therefore be written as:
\begin{equation}
    \mean\del{\T_\miss} 
        \approx 
        \mean\del{
            \cbr{
                \del{\T_{\miss,i},t_i}: \,i=1,\dotsc,N
            }
        }
        \equiv 
        \frac{1}{t_N - t_1} \sum_{i=1}^{N-1} \midpoint\del{\T_{\miss}, i} \Delta_i
\label{eq:weighted_mean}
\end{equation}
Applying this weighted mean formula to the temperatures measured at Waterloo Municipal Airport yields an annual mean of $\mean\del{\T_\miss} = 9.497\,\degreeC$ in 2015.

**Conventional estimate of mean temperature at Waterloo Airport**

![\label{fig:waterloo_mean_temp_by_hour}The 2015 mean temperature at Waterloo Municipal Airport conventionally estimated by averaging $\Tn$ and $\Tx$ depends on the hour of measurement.](../figures/waterloo_mean_temp_by_hour.png)

The conventional approach to estimating $\mean\del{\T_\miss}$ is to average the daily $\Tn$ and $\Tx$ measurements.
At Waterloo Airport, since we have $\T_\miss$, we can emulate this by first obtaining $\Tn\del{\T_\miss, \hour}$, $\Tx\del{\T_\miss, \hour}$ and then averaging the results together.
The result depends on $\hour$, and so I show the results as a function of $\hour$ in Figure \ref{fig:waterloo_mean_temp_by_hour}.
Strikingly, the estimate varies by over $1\,\degreeC$ depending on the hour of measurement.
If the $\Tn$/$\Tx$ measurements are made at 17:00 UTC, the average temperature will be estimated as $9.716\,\degreeC$.

**Imputation-based estimate of mean temperature at Waterloo Airport**

Now I go ahead and apply the $\mean$ function to the imputations.
This isn't quite so straightforward because for computational reasons I obtained the temperature imputations for 2015 in nine-day windows, with three days overlap between adjacent windows.
What follows is some mildly painful notation to make it completely clear how I'm obtaining my imputation-based estimate.
We denote the time interval covered by the $k^\text{th}$ window as $\window_k$, so $\cbr{i:\,t_i \in \window_k}$ is the indices of the observations that are within this window.
Each observation can fall in multiple windows.
For a given time $t$ and a window $k$, I can calculate the buffer $\buffer(t; \window_k) = \min\del{\max\del{\window_k} - t, t - \min\del{\window_k}}$ that separates the time $t$ and the window's closest edge.
The best window for imputing an observation at time $t$ is then found by $\argmax_k \del{\buffer\del{t; \window_k}}$.
And the set of indices of the midpoints (times halfway between two observations) for which window $k$ is the best is then written as:
\begin{equation}
\best_k \equiv
\cbr{
    i: \argmax_{k'} \del{\buffer\del{\midpoint\del{t, i}; \window_{k'}}} = k
}\,.
\end{equation}
Those midpoints occupy a duration of $\timespan\del{\best_k} = t_{\max\del{\best_k}+1} - t_{\min\del{\best_k}}$.
For the $l^\text{th}$ imputed time series $\T_\miss^{(l)}$ ($l=1,\dotsc,L$) within each window $k$, I obtain the weighted mean of imputed temperatures for the midpoints in the middle of the window:
\begin{equation}
    \overline{\T}_{k}^{(l)} = \mean\del{
            \cbr{
                \del{\T_{\miss,i}^{(l)},t_i} ;\,i \in \best_k
            }
        }
         \,,
\end{equation}
as defined in \eqref{eq:weighted_mean}.
The posterior mean and variance $\mu_k$, $\sigma^2_k$ are estimated by computing the sample mean and variance of $\overline{\T}_{k}^{(l)}$ over the $L$ imputations.
Then I combine the estimates for each window by taking a weighted mean of the $\mu_k$'s:
\begin{equation}
\begin{split}
    \E\del{\mean\del{\T_\miss} \mid \Tn, \Tx, \T_\obs, \hour} 
    &=
    \frac{1}{t_N - t_1} \sum_{k=1}^K \mu_k \timespan\del{\best_k} \\
    \var\del{\mean\del{\T_\miss} \mid \Tn, \Tx, \T_\obs, \hour} 
    &\approx \frac{1}{\del{t_N - t_1}^2} \sum_{k=1}^K \sigma^2_k \timespan\del{\best_k}^2 \\
\end{split}
\end{equation}
The expectation of the posterior of $\mean\del{\T_\miss}$ is exact up to Monte Carlo error, but the variance estimate ignores correlations between windows. 
Long story short, the resulting estimate is
\begin{equation}
\begin{split}
    \E\del{\mean\del{\T_\miss} \mid \Tn, \Tx, \T_\obs, \hour} 
    &=
    9.474\,\degreeC\\
    \var\del{\mean\del{\T_\miss} \mid \Tn, \Tx, \T_\obs, \hour} 
    &\approx \del{0.020\,\degreeC}^2\,, \\
\end{split}
\end{equation}
which is a good estimate of the true $\mean\del{\T_\miss} = 9.497\,\degreeC$.